# M1M3 LUT Verification
## Analysis Measured Forces vs LUT (no force balance!)

This notebook plots the LUT actuator forces in the M1M3 mirror and LUT vs measured forces.

We need to verify that M1M3 is applying the correct force balances in different stages of the tests. Part of this verification includes comparing the calculated elevation forces from the Elevation Look-Up Tables with the measured applied forces.  Remember that comparing this values with the applied forces only if the balance forces are turned off.   

## Notebook Setup

Setup input variables, import extensions and packages.

In [1]:
elevation_angle = 45.

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

from astropy.time import Time, TimeDelta
import asyncio
import glob
import os
import shlex
import subprocess
import sys

from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
import numpy as np
import pandas as pd
from pathlib import Path

from lsst_efd_client import EfdClient
from lsst.ts.idl.enums import MTM1M3
from lsst.ts.xml.tables.m1m3 import FATable as M1M3FATable
from lsst.sitcom import vandv

### What data can we retrieve from EFD?

In [3]:
client = EfdClient('usdf_efd')

## Calculate Elevation Forces 

In [5]:
fat = np.array(M1M3FATable.FATABLE)
ids = fat[:, M1M3FATable.FATABLE_ID]


# Create a Series emulating data format from the EFD
lut_elevation_x_forces = pd.Series(
    data=vandv.m1m3.lut_elevation_xforces(elevation_angle), 
    index=[f"xForces{i}" for i in range(ids.size)],
)

lut_elevation_y_forces = pd.Series(
    data=vandv.m1m3.lut_elevation_yforces(elevation_angle), 
    index=[f"yForces{i}" for i in range(ids.size)],
)

lut_elevation_z_forces = pd.Series(
    data=vandv.m1m3.lut_elevation_zforces(elevation_angle), 
    index=[f"zForces{i}" for i in range(ids.size)],
)

The LUT Elevation X forces should be all zero. 

## Display Elevation Forces

Here we just plot the LUT forces in each of the actuators at elevation 45.0 deg. Nothing else! No comparison with the applied forces at all.

In [ ]:
fig, axs = plt.subplots(
    num="Elevation Forces from Look-Up Table",
    nrows=1,
    ncols = 2,
    figsize=(12, 6),
    dpi=260)

fig.suptitle(f'LUT Forces at elevation {elevation_angle}')

vandv.m1m3.snapshot_zforces_overview(axs[0], lut_elevation_y_forces, prefix="yForces")
plt.title('Y forces')

vandv.m1m3.snapshot_zforces_overview(axs[1], lut_elevation_z_forces, prefix="zForces")
plt.title('Z forces')

plt.tight_layout()

## Comparison between LUT and measured applied forces

#### Retrieve data from EFD

In [7]:
start = Time('2023-05-30 10:15:0Z', scale='utc')
end = Time('2023-05-30 11:30:0Z', scale='utc') 
two_cycles = '2023-05-30 10:15:00Z'

In [8]:
xForce = [str("".join(("xForce",str(i)))) for i in range(156)]
yForce = [str("".join(("yForce",str(i)))) for i in range(156)]
zForce = [str("".join(("zForce",str(i)))) for i in range(156)]

df_all_x_forces = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    xForce, 
    start,
    end
)

df_all_y_forces = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    yForce, 
    start,
    end
)

df_all_z_forces = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    zForce, 
    start,
    end
)

In [ ]:
df_all_forces = pd.concat([df_all_x_forces.dropna(axis = 1), df_all_y_forces.dropna(axis = 1), df_all_z_forces.dropna(axis = 1)])
forces_resampled = df_all_forces.resample('1T').mean()

In [ ]:
df_static_x_forces = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    xForce, 
    start,
    end
)

df_static_y_forces = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    yForce, 
    start,
    end
)

df_static_z_forces = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    zForce, 
    start,
    end
)

In [ ]:
df_all_static_forces = pd.concat([df_static_x_forces.dropna(axis = 1), df_static_y_forces.dropna(axis = 1), df_static_z_forces.dropna(axis = 1)])
static_resampled = df_all_static_forces.resample('1T').mean()

In [ ]:
# Retrieve detailed state from system
df_state = await client.select_time_series(
    "lsst.sal.MTM1M3.logevent_detailedState", 
    "*", 
    start,
    end, 
)

df_state["detailedStateName"] = \
    df_state["detailedState"].map(lambda x: MTM1M3.DetailedState(x).name)

df_state = df_state.set_index("private_rcvStamp")
df_state.index = pd.to_datetime(df_state.index, unit="s")

In [ ]:
# Retrieve elevations
elevations = await client.select_time_series(
    'lsst.sal.MTMount.elevation',
    ['actualPosition', 'timestamp'],  
    start, 
    end,
)  
elevations = elevations['actualPosition'].resample('1T').mean()


#### Generate data when actuators were Active

In [ ]:
when_parked = df_state[df_state["detailedStateName"] == "PARKED"].index.tz_localize('UTC').tz_convert(forces_resampled.index.tz)
when_raising = df_state[df_state["detailedStateName"] == "RAISING"].index.tz_localize('UTC').tz_convert(forces_resampled.index.tz)
when_active = df_state[df_state["detailedStateName"] == "ACTIVE"].index.tz_localize('UTC').tz_convert(forces_resampled.index.tz)

forces_parked = forces_resampled.loc[(forces_resampled.index >= when_parked[0]) & (forces_resampled.index <= when_raising[0])]
el_parked = elevations.loc[(elevations.index >= when_parked[0]) & (elevations.index <= when_raising[0])]
forces_raising = forces_resampled.loc[(forces_resampled.index >= when_raising[0]) & (forces_resampled.index <= when_active[0])]
el_raising = elevations.loc[(elevations.index >= when_raising[0]) & (elevations.index <= when_active[0])]
forces_active = forces_resampled.loc[forces_resampled.index >= when_active[0]]
el_active = elevations.loc[elevations.index >= when_active[0]]

when_2cycles = pd.to_datetime(two_cycles, utc=True)
forces_2cycles = forces_resampled.loc[forces_resampled.index >= when_2cycles]
el_2cycles = elevations.loc[elevations.index >= when_2cycles]

#### Plot elevation and single actuator force time plot

In [ ]:
fig = plt.figure(figsize = (8,5))
elevations.plot()

plt.xlabel('Timestamp [UTC]')
plt.ylabel('Elevation (deg)')
l1 = plt.axvline(when_parked[0], lw="0.5", c="k")
l2 = plt.axvline(when_raising[0], lw="0.5", c="k", ls="--")
l3 = plt.axvline(when_active[0], lw="0.5", c="C1", ls="-")
plt.grid(":", lw=0.1)

fig.legend(
    [l1, l2, l3], 
    ["PARKED", "RAISING", "ACTIVE"], 
    ncols=4, 
    loc="upper right", 
    bbox_to_anchor=(0.75, 0.97)
)

These plots below correspond to the zForces for the actuator zForce0 at different elevations from 9:30 to 11:30

In [ ]:
fig = plt.figure(figsize = (13,5))

# Plot forces vs time for zForce[0]
plt.subplot(1,2,1)
plt.title('zForces0')
forces_active['zForce0'].plot(marker='.')
forces_parked['zForce0'].plot(marker='.', linestyle='--', color = 'gray')
forces_raising['zForce0'].plot(marker='.', linestyle='--', color = 'gray')

# Customize plot
plt.xlabel('Timestamp [UTC]')
plt.ylabel('Applied Force (N)')
plt.grid(":", lw=0.1)
# Add detailedState events
l1 = plt.axvline(when_parked[0], lw="0.5", c="k")
l2 = plt.axvline(when_raising[0], lw="0.5", c="k", ls="--")
l3 = plt.axvline(when_active[0], lw="0.5", c="C1", ls="-")
fig.legend(
    [l1, l2, l3], 
    ["PARKED", "RAISING", "ACTIVE"], 
    ncols=4, 
    loc="upper right", 
    bbox_to_anchor=(0.65, 1.0)
)

# Plot force vs elevation
plt.subplot(1,2,2)
plt.title('Force vs elevation')
plt.plot(el_active, forces_active['zForce0'], '.-', label = 'active')
plt.plot(el_parked, forces_parked['zForce0'], '.--', color='gray', label = 'not active')
plt.plot(el_raising, forces_raising['zForce0'], '.--', color='gray')

# Customize plot
plt.xlabel('Elevation (deg)')
plt.ylabel('Applied Force (N)')
plt.grid(":", lw=0.1)
plt.legend(
    bbox_to_anchor=(1.02, 1)
)

## Plotting LUT and measured Actuator forces as a function of elevation

Given the scale it is not possible to see how much do they differ, so we plot below the error as well. Keep scrolling!

In z axis one cannot appreciate it, but we go over the same elevation twice, so x axis plots show a cycle!

In [ ]:
# Get LUT elevation forces
elevations_lut = np.linspace(0, 90, 90)
lut_xforces = vandv.m1m3.lut_elevation_xforces(elevations_lut, as_array=True)
lut_yforces = vandv.m1m3.lut_elevation_yforces(elevations_lut, as_array=True)
lut_zforces = vandv.m1m3.lut_elevation_zforces(elevations_lut, as_array=True)
labels_z = [f"zForce{i}" for i in range(ids.size)]
labels_y = [f"yForce{i}" for i in range(ids.size)]
labels_x = [f"xForce{i}" for i in range(ids.size)]


#### Z axis

In [ ]:
fig = plt.figure(figsize = (15,120))
for idx in range(156):
    plt.subplot(40,4,idx + 1)
    plt.plot(elevations_lut, lut_zforces[idx], '-', label='LUT')
    plt.plot(el_active, forces_active[f"zForce{idx}"], '-', label='Actuators')
    plt.title(labels_z[idx])
    plt.ylabel('Force (N)')
    plt.xlabel('Elevation (deg)')
    plt.legend()
plt.tight_layout()


#### Y axis

At the time of the making of these plots, it is unclear to me if the data that I am retrieveing from the LUT is correct for y axis. It seems that for some actuators the lut is zero when it shouldn't be. 

In [ ]:
fig = plt.figure(figsize = (15,75))
for idx in range(100):
    plt.subplot(25,4,idx + 1)
    plt.plot(elevations_lut, lut_yforces[idx], '-', label='LUT')
    plt.plot(el_active, forces_active[f"yForce{idx}"], '-', label='Actuators')
    plt.title(labels_y[idx])
    plt.ylabel('Force (N)')
    plt.xlabel('Elevation (deg)')
    plt.legend()
plt.tight_layout()

#### X axis

Here we are always very close to zero, given the scale of the subplots.

In [ ]:
fig = plt.figure(figsize = (15,16))
for idx in range(12):
    plt.subplot(5,4,idx + 1)
    plt.plot(elevations_lut, lut_xforces[idx], '-', label='LUT')
    plt.plot(el_active, forces_active[f"xForce{idx}"], '-', label='Actuators')
    plt.title(labels_x[idx])
    plt.ylabel('Force (N)')
    plt.xlabel('Elevation (deg)')
    plt.legend()
plt.tight_layout()

### LUT - Actuator Force Error

These plots show the difference for z, y and x axis actuator forces. It shows the differences from 10:15 UTC to 11:30 UTC, which corresponds to 90 -> 0 -> 90 deg.

In [ ]:
lut_xforces = vandv.m1m3.lut_elevation_xforces(el_2cycles, as_array=True)
lut_yforces = vandv.m1m3.lut_elevation_yforces(el_2cycles, as_array=True)
lut_zforces = vandv.m1m3.lut_elevation_zforces(el_2cycles, as_array=True)

plt.figure(figsize = (15,120))
for idx in range(156):
    plt.subplot(40,4,idx + 1)
    plt.plot(el_2cycles, lut_zforces[idx] - forces_2cycles[f"zForce{idx}"], '-')
    plt.title(labels_z[idx])
    plt.ylabel('Force Difference (N)')
    plt.xlabel('Elevation (deg)')
    plt.grid(":", lw=0.1)
plt.tight_layout()

In [ ]:
lut_yforces = vandv.m1m3.lut_elevation_yforces(el_2cycles, as_array=True)

plt.figure(figsize = (15,75))
for idx in range(100):
    plt.subplot(25,4,idx + 1)
    plt.plot(el_2cycles, lut_yforces[idx] - forces_2cycles[f"yForce{idx}"], '-')
    plt.title(labels_y[idx])
    plt.ylabel('Force Difference (N)')
    plt.xlabel('Elevation (deg)')
    plt.grid(":", lw=0.1)
plt.tight_layout()

In [ ]:
lut_xforces = vandv.m1m3.lut_elevation_xforces(el_2cycles, as_array=True)

plt.figure(figsize = (15,15))
for idx in range(12):
    plt.subplot(5,4,idx + 1)
    plt.plot(el_2cycles, lut_xforces[idx] - forces_2cycles[f"xForce{idx}"], '-')
    plt.title(labels_x[idx])
    plt.ylabel('Force Difference (N)')
    plt.xlabel('Elevation (deg)')
    plt.grid(":", lw=0.1)
plt.tight_layout()